In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import os
import re
import scanpy as sc
from scDesign2.model_fitting import fit_model_scDesign2

Randomly sample 5000 mouse genes

In [3]:
y = pd.read_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/mouse.csv', sep=',', index_col=0) #on_bad_lines='skip' the first row
y.columns = [re.sub(r"\.\d+$", '', string) for string in y.columns]
y.columns = [name.split('_', 2)[-1] for name in y.columns]
nonspikes = ~y.index.str.contains('ercc', case=False)
y = y.loc[nonspikes, :]

mask_nonzero = y != 0
row_means = mask_nonzero.mean(axis=1)

selected_rows = row_means.sample(n=5000, random_state=42).index
y = y.loc[selected_rows]

y.to_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/mouse_5000_random.csv', sep=',', index=True)


cell_type_sel = ['Stem', 'Goblet', 'Tuft', 'TA.Early', 'Enterocyte.Progenitor', 'Enterocyte.Progenitor.Early']
copula_result = fit_model_scDesign2(y, cell_type_sel, sim_method = 'copula', jitter=True, ncores=1)
copula_result['Macro'] = copula_result['Stem']
copula_result['Plasma'] = copula_result['Enterocyte.Progenitor.Early']
copula_result['program1'] = copula_result['Tuft']
copula_result['program2'] = copula_result['Goblet']
copula_result['program3'] = copula_result['Enterocyte.Progenitor']
copula_result.pop('Stem')
copula_result.pop('Enterocyte.Progenitor.Early')
copula_result.pop('Tuft')
copula_result.pop('Goblet')
copula_result.pop('Enterocyte.Progenitor')
with open('copula_result_mouse_5000_random.pkl', 'wb') as outp:
	pickle.dump(copula_result, outp, pickle.HIGHEST_PROTOCOL)

Randomly sample 5000 breast genes

In [3]:
temp = sc.read_h5ad(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/bcells.h5ad')
temp = temp[temp.obs['disease'] == 'normal']
x = list(temp.obs['cell_type'].values)
mat = temp.raw.X
mat.index = x
z = pd.DataFrame(mat.toarray(), index=x, columns=temp.var_names)
z.index = x
z = z.T

z = z.sample(n=5000)
z.to_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/bcells_5000_random.csv', sep=',', index=True)

cell_type_sel = ['naive B cell', 'unswitched memory B cell', 'class switched memory B cell', 'IgA plasma cell', 'IgG plasma cell']
copula_result = fit_model_scDesign2(z, cell_type_sel, sim_method = 'copula', jitter=True, ncores=1)
copula_result['Macro'] = copula_result['IgA plasma cell']
copula_result['Plasma'] = copula_result['IgG plasma cell']
copula_result['program1'] = copula_result['naive B cell']
copula_result['program2'] = copula_result['unswitched memory B cell']
copula_result['program3'] = copula_result['class switched memory B cell']
copula_result.pop('IgA plasma cell')
copula_result.pop('IgG plasma cell')
copula_result.pop('naive B cell')
copula_result.pop('unswitched memory B cell')
copula_result.pop('class switched memory B cell')

with open('copula_result_bcell_5000_random.pkl', 'wb') as outp:
	pickle.dump(copula_result, outp, pickle.HIGHEST_PROTOCOL)

Selectively choose 5000 breast genes

In [17]:
temp = sc.read_h5ad(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/bcells.h5ad')
# temp = temp[temp.obs['disease'] == 'normal']
x = list(temp.obs['cell_type'].values)
mat = temp.raw.X
mat.index = x
z = pd.DataFrame(mat.toarray(), index=x, columns=temp.var_names)
z.index = x
z = z.T


mask_nonzero = z != 0
row_means = mask_nonzero.mean(axis=1)
selected_zero_rows = row_means[row_means < 0.1].sample(n=1350, random_state=42).index


# selected_zero_rows = row_means[row_means < 0.1].sample(n=1550, random_state=42).index
# selected_nonzero_rows = row_means[(row_means >= 0.1) & (row_means < 0.2)].sample(n=1800, random_state=42).index
# selected_rows_0_2 = row_means[(row_means >= 0.2) & (row_means < 0.5)].sample(n=1200, random_state=42).index

selected_nonzero_rows = row_means[(row_means >= 0.1) & (row_means < 0.2)].sample(n=1900, random_state=42).index
selected_rows_0_2 = row_means[(row_means >= 0.2) & (row_means < 0.5)].sample(n=1400, random_state=42).index
selected_rows_0_5 = row_means[(row_means >= 0.5) & (row_means < 0.9)].sample(n=250, random_state=42).index
selected_rows_0_9 = row_means[row_means >= 0.9].sample(n=100, random_state=42).index
selected_rows = np.concatenate([selected_rows_0_9, selected_rows_0_5, selected_rows_0_2, selected_nonzero_rows, selected_zero_rows])

z = z.loc[selected_rows]
z.to_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/bcells_5000_chosen.csv', sep=',', index=True)

cell_type_sel = ['naive B cell', 'unswitched memory B cell', 'class switched memory B cell', 'IgA plasma cell', 'IgG plasma cell']
copula_result = fit_model_scDesign2(z, cell_type_sel, sim_method = 'copula', jitter=True, ncores=1)
copula_result['Macro'] = copula_result['IgA plasma cell']
copula_result['Plasma'] = copula_result['IgG plasma cell']
copula_result['program1'] = copula_result['naive B cell']
copula_result['program2'] = copula_result['unswitched memory B cell']
copula_result['program3'] = copula_result['class switched memory B cell']
copula_result.pop('IgA plasma cell')
copula_result.pop('IgG plasma cell')
copula_result.pop('naive B cell')
copula_result.pop('unswitched memory B cell')
copula_result.pop('class switched memory B cell')

with open('copula_result_bcell_5000_chosen.pkl', 'wb') as outp:
	pickle.dump(copula_result, outp, pickle.HIGHEST_PROTOCOL)

Generate copula_result for bcells

In [6]:
data = pd.read_csv(os.path.dirname(os.path.abspath('')) + '/SplatterSim-mainScDesign2/scDesign2/count_matrices/bcells_100_250_1400_1900_1350_09_05_02_01.csv', sep=',', index_col=0) #on_bad_lines='skip' the first row
data.columns = [re.sub(r"\.\d+$", "", string) for string in data.columns]

unique_cell_type = np.unique(data.columns)

# cell_type_sel = ['naive B cell', 'unswitched memory B cell', 'IgG plasma cell', 'class switched memory B cell', 'IgA plasma cell']
cell_type_sel = ['Macro', 'Plasma', 'program1', 'program2', 'program3']


copula_result = fit_model_scDesign2(data, cell_type_sel, sim_method = 'copula', jitter=True)

# copula_result['Macro'] = copula_result['IgG plasma cell']
# copula_result['Plasma'] = copula_result['IgA plasma cell']
# copula_result['program1'] = copula_result['unswitched memory B cell']
# copula_result['program2'] = copula_result['naive B cell']
# copula_result['program3'] = copula_result['class switched memory B cell']
# copula_result.pop('unswitched memory B cell')
# copula_result.pop('naive B cell')
# copula_result.pop('class switched memory B cell')
# copula_result.pop('IgG plasma cell')
# copula_result.pop('IgA plasma cell')

with open(os.path.dirname(os.path.abspath('')) + '/SplatterSim-mainScDesign2/notebook/data/copula_result_bcells_100_250_1400_1900_1350_09_05_02_01_test.pkl', 'wb') as outp:
	pickle.dump(copula_result, outp, pickle.HIGHEST_PROTOCOL)

Generate copula_result for mouse

In [ ]:
data = pd.read_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/mouse_800_3200_1000_1_05_01.csv', sep=',', index_col=0) #on_bad_lines='skip' the first row
data.columns = [re.sub(r"\.\d+$", "", string) for string in data.columns]

unique_cell_type = np.unique(data.columns)

cell_type_sel = ["Stem", "Enterocyte.Progenitor.Early", "Tuft", "Goblet", "Enterocyte.Progenitor"]

copula_result = fit_model_scDesign2(data, cell_type_sel, sim_method = 'copula', jitter=True)

copula_result['Macro'] = copula_result['Stem']
copula_result['Plasma'] = copula_result['Enterocyte.Progenitor.Early']
copula_result['program1'] = copula_result['Tuft']
copula_result['program2'] = copula_result['Goblet']
copula_result['program3'] = copula_result['Enterocyte.Progenitor']
copula_result.pop('Stem')
copula_result.pop('Enterocyte.Progenitor.Early')
copula_result.pop('Tuft')
copula_result.pop('Goblet')
copula_result.pop('Enterocyte.Progenitor')

with open(os.path.dirname(os.path.abspath('')) + '/notebook/data/copula_result_mouse_800_3200_1000_1_05_01.pkl', 'wb') as outp:
	pickle.dump(copula_result, outp, pickle.HIGHEST_PROTOCOL)

Selectively sample 5000 genes

In [ ]:
# temp = scanpy.read_h5ad(os.path.dirname(os.path.abspath('')) + '/scDesign2/bcells.h5ad')
# x = list(temp.obs['cell_type'].values)
# mat = temp.raw.X
# mat.index = x
# y = pd.DataFrame(mat.toarray(), index=x, columns=temp.var_names)
# y.index = x
# y = y.T

y = pd.read_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/mouse.csv', sep=',', index_col=0) #on_bad_lines='skip' the first row
y.columns = [re.sub(r"\.\d+$", "", string) for string in y.columns]
y.columns = [name.split('_', 2)[-1] for name in y.columns]

mask_nonzero = y != 0
row_means = mask_nonzero.mean(axis=1)

selected_zero_rows = row_means[row_means < 0.1].sample(n=1350, random_state=42).index
# selected_nonzero_rows = row_means[(row_means >= 0.1) & (row_means < 0.2)].sample(n=1900, random_state=42).index
# selected_rows_0_2 = row_means[(row_means >= 0.2) & (row_means < 0.5)].sample(n=1400, random_state=42).index
# selected_rows_0_5 = row_means[(row_means >= 0.5) & (row_means < 0.9)].sample(n=250, random_state=42).index
# selected_rows_0_9 = row_means[row_means >= 0.9].sample(n=100, random_state=42).index


selected_rows = np.concatenate([selected_rows_0_2, selected_nonzero_rows, selected_zero_rows])
y = y.loc[selected_rows]

y.to_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/mouse_800_3200_1000_1_05_01.csv', sep=',', index=True)

Differential anallysis

In [ ]:
import os
import pandas as pd
import re
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv(os.path.dirname(os.path.abspath('')) + '/scDesign2/count_matrices/bcells_100_250_1400_1900_1350_09_05_02_01.csv', sep=',', index_col=0) #on_bad_lines='skip' the first row
data.columns = [re.sub(r"\.\d+$", "", string) for string in data.columns]
adata = sc.AnnData(X=data.values.T, var=pd.DataFrame(index=data.index), obs=pd.DataFrame(index=data.columns))

adata.obs['cell_type'] = data.columns
# Normalize the counts.
sc.pp.normalize_total(adata, target_sum=1e4)
# Logarithmize the data.
sc.pp.log1p(adata)
adata.raw = adata
all_groups = ['unswitched memory B cell', 'naive B cell', 'class switched memory B cell', 'IgG plasma cell', 'IgA plasma cell']
sc.tl.rank_genes_groups(adata, 'cell_type', groups=all_groups, method='wilcoxon')
groups=['IgG plasma cell', 'IgA plasma cell']
# Extract the results.
significant_genes_total = []
for group in groups:
    names = adata.uns['rank_genes_groups']['names'][group]
    logfoldchanges = adata.uns['rank_genes_groups']['logfoldchanges'][group]
    pvals = adata.uns['rank_genes_groups']['pvals'][group]
    pvals_adj = adata.uns['rank_genes_groups']['pvals_adj'][group]

    # Combine the above arrays into one results dataframe.
    results_df = pd.DataFrame({
        'names': names,
        'logfoldchanges': logfoldchanges,
        'pvals': pvals,
        'pvals_adj': pvals_adj
    })

    # Filter genes by adjusted p-value (you may adjust the threshold)
    df_significant = results_df.loc[(results_df['pvals_adj'] < 0.05) & (np.abs(results_df['logfoldchanges']) > 1.5), :]

    # Create a new column for -log10 of the adjusted p-value
    df_significant['neg_log_pvals_adj'] = -np.log10(df_significant['pvals_adj'])

    # Create the volcano plot
    plt.figure(figsize=(10, 5))
    plt.scatter(df_significant['logfoldchanges'], df_significant['neg_log_pvals_adj'], c='blue')
    plt.title(f'Volcano plot of differentially expressed genes in {group}')
    plt.xlabel('Log2 fold change')
    plt.ylabel('-Log10 adjusted p-value')
    plt.show()


    # Get the names of the significant genes
    significant_genes = df_significant['names'].tolist()
    print(f'Significant genes for {group}:')
    significant_genes_total += significant_genes

# Convert row indices to a set to remove duplicates
row_indices = list(set(significant_genes_total))

# Create a boolean mask based on the row indices
mask = data.index.isin(row_indices)

# Set rows with the specified indices to 0
data.loc[mask] = 0

sc.pl.rank_genes_groups_dotplot(adata, n_genes=100, groupby='cell_type')

# data.to_csv(os.path.dirname(os.path.abspath('')) + '/SplatterSim-mainScDesign2/scDesign2/bcells_1400_1900_1700_05_02_01_de_plasma.csv', sep=',')